In [ ]:
import torch
from src.task_vectors import TaskVector
from src.eval import eval_single_dataset, evaluate
from src.args import parse_arguments
import os

args = parse_arguments()

args.train_dataset = args.dataset_name + "Val"
args.eval_datasets = [args.dataset_name]
args.batch_size = 64

# Config
# datasets = ['MNISTVal']
datasets_finetuned_name = {
    args.apply_task_vectors[i]: f"finetuned_full_seed_{args.manual_seed}.pt"
    for i in range(len(args.apply_task_vectors))
}

args.data_location = "./data"
model = args.model
args.save = f"checkpoints/{model}"
pretrained_checkpoint = f"checkpoints/{model}/zeroshot.pt"

# Create the task vectors
task_vectors = [
    TaskVector(pretrained_checkpoint, f"checkpoints/{model}/{dataset}/{datasets_finetuned_name[dataset]}")
    for dataset in args.apply_task_vectors
]

# Sum the task vectors
task_vector_sum = sum(task_vectors)

# Apply the resulting task vector
image_encoder = task_vector_sum.apply_to(pretrained_checkpoint, scaling_coef=args.tv_scaling_coeff)

# Evaluate
evaluate(image_encoder, args)

# Evaluate # NOTE this evaluates on the training set, not the validation set
# for dataset in args.apply_task_vectors:
#     eval_single_dataset(image_encoder, dataset, args)
#     pass

# Save model with applied task vectors
# datasets_str = '_and_'.join(args.apply_task_vectors)
# export_path = f'checkpoints/{model}/{datasets_str}'
# os.makedirs(export_path, exist_ok=True)
# if args.save is not None:
#     image_encoder.save(f"{export_path}/finetuned_full_seed_{args.manual_seed}_scaling_coeff_{args.tv_scaling_coeff}.pt")